<h1 align="center">KRX Big-Data Contest</h1>

# [ 1 ] Overview

### 1. Sources

#### - Basic

 - `[유가증권]일별 시세정보(주문번호-1300-27)` : 2020년 1분기 주식 정보 -> _CSV 형식_
 - `[유가증권]일별 시세정보(주문번호-1300-30)` : 2021년 1분기 주식 정보 -> _CSV 형식_
 - `[유가증권]일별 시세정보(주문번호-1300-33)` : 2022년 1분기 주식 정보 -> _CSV 형식_

#### - Extension ( https://kr.investing.com )

 - `환율 추이` : 2020년, 2021년, 2022년 1분기 -> _CSV 형식_
 - `미국 3년 채권수익률` : 2020년, 2021년, 2022년 1분기 -> _CSV 형식_

```txt
https://kr.investing.com/robots.txt
크롤링 정책상 Disallow를 피함
```


<br><br><br>

### 2. Targets from `.CSV` files (Input)

| 항목 명 | 항목 영어명 | 모델 학습값 여부 |
|:---:|:---:|:---:|
|`거래일자`|TRD_DD|Y|
|`종목코드`|ISU_CD|N|
|`종목명`|ISU_NM|N|
|`시가`|OPNPRC|Y|
|`고가`|HGPRC|Y|
|`저가`|LWPRC|Y|
|`종가`|CLSPRC|Y|
|(누적)`거래량`|ACC_TRDVOL|Y|
|`업종구분`(지수업종코드)|IDX_IND_CD|N|
|`PER`(주가수익률)|PER|Y|
|`상장일`|LIST_DD|N|
|`시가총액`|MKTCAP|Y|

<!-- <br><br><br>

### 3. Results (Output)

| Property | Description |
|:---:|:---:|
|TRD_DD|`거래일자`|
|ISU_CD|`종목코드`|
|ISU_NM|`종목명`|
|OPNPRC|`시가`|
|HGPRC|`고가`|
|LWPRC|`저가`|
|CLSPRC|`종가`|
|ACC_TRDVOL|(누적)`거래량`|
|IDX_IND_CD|`업종구분`(지수업종코드)|
|PER|`PER`(주가수익률)|
|LIST_DD|`상장일`|
|MKTCAP|`시가총액`|
 -->
<br><br><br><hr>

# [ 2 ] Importing Modules

In [1]:
# Data Handlers
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display

# Code Libraries
import os
import copy
import abc

<br><br><br>

# [ 3 ] Declarations

In [2]:
############################################################################################################

import copy

class Utils:
    """
    데이터 처리를 위해 유용한 기능들을 정의한 클래스입니다.
    """
    @staticmethod
    def generate_int_range(start:int, end:int)->iter:
        """
        start와 end 사이의 정수들을 반환하는 Generator입니다.
        """
        if(start >= end):
            raise SyntaxError(f"{start} must be larger than {end}")
        while start <= end:
            yield start
            start += 1
            
    @staticmethod
    def clone(target:object)->object:
        """
        깊은 복사를 한 인스턴스를 반환합니다.
        """
        return copy.deepcopy(target)
    
############################################################################################################

import pandas

class PandasBasedCSVHandler:
    """
    Pandas 모듈을 기반으로 CSV 파일 데이터를 다루는 클래스입니다.
    """
    def __init__(self, handler:pandas):
        self.__handler = handler
        self.__data = dict()
        
    @property
    def handler(self)->pandas:
        """
        주입받은 Pandas 객체를 반환합니다.
        이미 생성된 handler 인스턴스는 대체될 수 없고 반환만 가능합니다.
        """
        return self.__handler
    
    def take_data_from_CSV_file(self, *, data_id:object, filepath:str, encoding:str="utf-8")->object:
        """
        불러올 CSV 파일의 경로를 받아 데이터를 가져오고
        데이터를 식별할 data_id를 받습니다.
        Argument를 반드시 키워드로 명시하여 Parameter에 전달해야 합니다.
        """
        self.__data[data_id] = self.__handler.read_csv(filepath, encoding=encoding)
            
        return self
    
    def get_CSV_data(self)->dict:
        """
        다음 Dictionary 자료구조를 반환합니다.
        key의 타입(자료형)은 정수형으로 의도되었지만 어떤 타입이 들어올지는 자유입니다.
        value는 Pandas 타입의 객체입니다.
        """
        return self.__data
    
    def validate(self, target_properties:list)->object:
        """
        모든 Pandas 데이터가 target_properties에 명시된 속성을 가지고 있는지 확인합니다.
        만일, 속성이 매칭되지 않으면 예외가 발생할 것입니다.
        모든 과정이 성공하면 True를 반환합니다.
        """
        for data_key, _ in self.__data.items():
            self.__data[data_key][target_properties]
        return True

############################################################################################################
import datetime
import requests
import json
from pandas import json_normalize

class KRXStockCrawler:
    """
    """
    def __init__(self):
        self.__url = "http://data.krx.co.kr/comm/bldAttendant/getJsonData.cmd"
        self.__headers = "User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36"
        self.__base_data = {
            "bld": "dbms/MDC/STAT/standard/MDCSTAT01701",
            "locale": "ko_KR",
            "param1isuCd_finder_stkisu0_1": "ALL",
            "share": 1,
            "money": 1,
            "csvxls_isNo": "false"
        }
        
    def target(self, code:str, start_date:int, end_date:int)->dict:
        """
        """
        query_data = {
            "isuCd": code,
            "strtDd": start_date,
            "endDd": end_date,
        }
        res = requests.post('http://data.krx.co.kr/comm/bldAttendant/getJsonData.cmd', data=dict(**self.__base_data,**query_data))
        dict_json = json.loads(res.text)
        return dict_json['output']
    
############################################################################################################
class KRXStockData:
    """
    
    """
    pass
    

############################################################################################################

class DataVisualization:
    pass

############################################################################################################

<br><br><br>

# [ 4 ] Stock Predictions

In [26]:
# "종목코드"를 기준으로 각 CSV 데이터들을 식별하는 용도의 자료구조
index_properties:list = dict()

## 01-A. 데이터 수집 및 전처리 단계: `KRX에서 기본적으로 제공받은 CSV`

In [27]:
# class<PandasBasedCSVHandler> 인스턴스 생성
csv_handler_krx:PandasBasedCSVHandler = PandasBasedCSVHandler(pd)

### (1) 데이터 경로 및 이름 설정

In [28]:
# CSV 파일 루트 경로
root_dir:str = os.path.join("..", "data")

# CSV 파일 전체 경로 및 이름 형식
filepath_form:str = os.path.join(root_dir, "{0}","{0}_{1}.csv")

# 각 CSV 파일들을 가져오기 위한 프로파일 list<dict[]>
csv_file_profiles : list = [
    {
        "name" : "[유가증권]일별 시세정보(주문번호-1300-27)",
        "date_range" : [202001, 202003]
    },
    {
        "name" : "[유가증권]일별 시세정보(주문번호-1300-30)",
        "date_range" : [202101, 202103]
    },
    {
        "name" : "[유가증권]일별 시세정보(주문번호-1300-33)",
        "date_range" : [202201, 202203]
    }
]


# 데이터 전처리에 필요한 속성들
selected_properties = ["거래일자","종목코드","종목명", "시가", "고가", "저가", "종가", "거래량", "업종구분", "PER", "상장일", "시가총액"]

### (2) CSV 파일 데이터 불러오기

In [29]:
# CSV 프로파일 기반으로 class<PandasBasedCSVHandler> 인스턴스에 데이터 병합
for csv_file_info in csv_file_profiles: # 프로파일 요소 기반 iteration 작업
    for date_num in Utils.generate_int_range(csv_file_info["date_range"][0], csv_file_info["date_range"][1]): # Iterator 생성
        csv_handler_krx.take_data_from_CSV_file(
            data_id = date_num, # Year + Month 형식의 정수
            filepath = filepath_form.format(csv_file_info["name"], date_num), # 폴더를 포함한 전체 경로의 파일명
            encoding="cp949" # 파일 인코딩 명시
        )

 - 날짜 형식 일치시키기

In [31]:
dataframe_list = csv_handler_krx.get_CSV_data()

for df_id in dataframe_list:
    dataframe_list[df_id]["거래일자"] = pd.to_datetime(dataframe_list[df_id]['거래일자'].astype('str'))  

In [32]:
# 검증할 속성 내용들은 [1. 데이터 경로 및 이름 설정]의 selected_properties 변수 참고
csv_handler_krx.validate(selected_properties)

True

#### - 핸들러(Hander)에 저장된 CSV 데이터 형식

```js
{
    202001 : Pandas,
    202002 : Pandas,
    202003 : Pandas,
    202101 : Pandas,
    202102 : Pandas,
    202103 : Pandas,
    202201 : Pandas,
    202202 : Pandas,
    202203 : Pandas
}
```

## 01-B. 데이터 수집 및 전처리 단계: `미국 3년 채권수익률`

In [33]:
# class<PandasBasedCSVHandler> 인스턴스 생성
csv_handler_bond:PandasBasedCSVHandler = PandasBasedCSVHandler(pd)

### (1) 데이터 경로 및 이름 설정

In [34]:
# CSV 파일 루트 경로
root_dir:str = os.path.join("..", "data")

# CSV 파일 전체 경로 및 이름 형식
filepath_form:str = os.path.join(root_dir, "{0}","{1}.csv")

# 각 CSV 파일들을 가져오기 위한 프로파일 list<dict[]>
csv_file_profiles : list = [
    {
        "name" : "미국 3년 채권수익률",
        "date_range" : [202001, 202003]
    },
    {
        "name" : "미국 3년 채권수익률",
        "date_range" : [202101, 202103]
    },
    {
        "name" : "미국 3년 채권수익률",
        "date_range" : [202201, 202203]
    }
]

# 데이터 전처리에 필요한 속성들
selected_properties = ["날짜", "종가", "오픈", "고가", "저가", "변동 %"]

### (2) CSV 파일 데이터 불러오기

In [35]:
# CSV 프로파일 기반으로 class<PandasBasedCSVHandler> 인스턴스에 데이터 병합
for csv_file_info in csv_file_profiles: # 프로파일 요소 기반 iteration 작업
    for date_num in Utils.generate_int_range(csv_file_info["date_range"][0], csv_file_info["date_range"][1]): # Iterator 생성
        csv_handler_bond.take_data_from_CSV_file(
            data_id = date_num, # Year + Month 형식의 정수
            filepath = filepath_form.format(csv_file_info["name"], date_num), # 폴더를 포함한 전체 경로의 파일명
            encoding="cp949" # 파일 인코딩 명시
        )

 - 날짜 형식 일치시키기

In [36]:
dataframe_list = csv_handler_bond.get_CSV_data()

for df_id in dataframe_list:
    dataframe_list[df_id]['날짜'] = pd.to_datetime(dataframe_list[df_id]['날짜'], format='%Y년 %m월 %d일')  

### (3) 데이터 속성 검증하기

In [37]:
# 검증할 속성 내용들은 [1. 데이터 경로 및 이름 설정]의 selected_properties 변수 참고
csv_handler_bond.validate(selected_properties)

True

#### - 핸들러(Hander)에 저장된 CSV 데이터 형식

```js
{
    202001 : Pandas,
    202002 : Pandas,
    202003 : Pandas,
    202101 : Pandas,
    202102 : Pandas,
    202103 : Pandas,
    202201 : Pandas,
    202202 : Pandas,
    202203 : Pandas
}
```

## 01-C. 데이터 수집 및 전처리 단계: `환율 추이`

In [38]:
# class<PandasBasedCSVHandler> 인스턴스 생성
csv_handler_exchange_rate:PandasBasedCSVHandler = PandasBasedCSVHandler(pd)

### (1) 데이터 경로 및 이름 설정

In [39]:
# CSV 파일 루트 경로
root_dir:str = os.path.join("..", "data")

# CSV 파일 전체 경로 및 이름 형식
filepath_form:str = os.path.join(root_dir, "{0}","{1}.csv")

# 각 CSV 파일들을 가져오기 위한 프로파일 list<dict[]>
csv_file_profiles : list = [
    {
        "name" : "환율 추이",
        "date_range" : [202001, 202003]
    },
    {
        "name" : "환율 추이",
        "date_range" : [202101, 202103]
    },
    {
        "name" : "환율 추이",
        "date_range" : [202201, 202203]
    }
]

# "종목코드"를 기준으로 각 CSV 데이터들을 식별하는 용도의 자료구조
index_properties:list = dict()

# 데이터 전처리에 필요한 속성들
selected_properties = ["날짜", "종가", "오픈", "고가", "저가", "변동 %"]

### (2) CSV 파일 데이터 불러오기

In [40]:
# CSV 프로파일 기반으로 class<PandasBasedCSVHandler> 인스턴스에 데이터 병합
for csv_file_info in csv_file_profiles: # 프로파일 요소 기반 iteration 작업
    for date_num in Utils.generate_int_range(csv_file_info["date_range"][0], csv_file_info["date_range"][1]): # Iterator 생성
        csv_handler_exchange_rate.take_data_from_CSV_file(
            data_id = date_num, # Year + Month 형식의 정수
            filepath = filepath_form.format(csv_file_info["name"], date_num), # 폴더를 포함한 전체 경로의 파일명
            encoding="cp949" # 파일 인코딩 명시
        )

 - 날짜 형식 일치시키기

In [41]:
dataframe_list = csv_handler_exchange_rate.get_CSV_data()

for df_id in dataframe_list:
    dataframe_list[df_id]['날짜'] = pd.to_datetime(dataframe_list[df_id]['날짜'], format='%Y년 %m월 %d일')  

In [42]:
# 검증할 속성 내용들은 [1. 데이터 경로 및 이름 설정]의 selected_properties 변수 참고
csv_handler_exchange_rate.validate(selected_properties)

True

#### - 핸들러(Hander)에 저장된 CSV 데이터 형식

```js
{
    202001 : Pandas,
    202002 : Pandas,
    202003 : Pandas,
    202101 : Pandas,
    202102 : Pandas,
    202103 : Pandas,
    202201 : Pandas,
    202202 : Pandas,
    202203 : Pandas
}
```

In [43]:
csv_handler_krx.get_CSV_data()[202001].tail()

,거래일자,시장ID,종목코드,종목명,시가,고가,저가,종가,거래량,거래대금,...,주문량기준취소율,종목별거래횟수기준매수매도불균형,종목별거래량기준매수매도불균형,주문횟수기준주문불균형,주문량기준주문불균형,평균호가스프레드,평균비율스프레드,평균유효스프레드,평균실현스프레드,HS 역선택비용
18322,2020-01-23,STK,KYG5307W1015,엘브이엠씨홀딩스보통주,4220,4325,4200,4290,178258,757659915,...,0.209617,0.591862,0.444226,1.415942,1.016236,15.250000,0.310432,16.331689,3.841060,12.490629
18323,2020-01-28,STK,KYG5307W1015,엘브이엠씨홀딩스보통주,4050,4200,3740,4095,348080,1404993900,...,0.223843,0.377031,0.286203,1.792737,0.924008,60.220571,0.547794,35.148459,14.807236,20.341223
18324,2020-01-29,STK,KYG5307W1015,엘브이엠씨홀딩스보통주,4095,4180,4095,4105,104726,433111520,...,0.209451,0.546139,0.447224,1.129568,0.758050,13.241726,0.316197,15.988701,10.753968,5.234733
18325,2020-01-30,STK,KYG5307W1015,엘브이엠씨홀딩스보통주,4100,4145,3985,4050,145074,586088370,...,0.258182,0.375830,0.311054,1.687097,0.904437,17.125451,0.387536,19.621514,10.398860,9.222654
18326,2020-01-31,STK,KYG5307W1015,엘브이엠씨홀딩스보통주,4000,4250,4000,4085,193650,797499085,...,0.245505,0.532700,0.406796,1.050000,0.873936,18.695652,0.383883,20.617089,-0.902857,21.519946


In [44]:
csv_handler_bond.get_CSV_data()[202001].tail()

,날짜,종가,오픈,고가,저가,변동 %
21,2020-01-06,1.5593,1.5270,1.5760,1.5160,2.32%
22,2020-01-05,1.5240,1.5267,1.5267,1.5240,-1.40%
23,2020-01-03,1.5457,1.5620,1.5730,1.5240,-3.09%
24,2020-01-02,1.5950,1.6140,1.6250,1.5650,-0.85%
25,2020-01-01,1.6086,1.6086,1.6086,1.6086,0.00%


In [45]:
csv_handler_exchange_rate.get_CSV_data()[202001].tail()

,날짜,종가,오픈,고가,저가,변동 %
18,2020-01-07,"1,167.30","1,167.54","1,168.82","1,163.11",0.03%
19,2020-01-06,"1,166.94","1,165.89","1,172.99","1,165.78",0.17%
20,2020-01-03,"1,164.95","1,157.94","1,168.83","1,155.70",0.66%
21,2020-01-02,"1,157.35","1,155.02","1,161.15","1,153.48",0.29%
22,2020-01-01,"1,154.02","1,155.07","1,155.32","1,154.08",0.00%
